In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AdQt8qjDPnoE9RxzJaolwJf79oADzFx07_e1Lu43vgiRttFUtuOIgEFU8VU

Successfully saved authorization token.


In [2]:
import os
import datetime
from datetime import timedelta
import pandas as pd

In [3]:
# !pip install geemap
import ee
import geemap
Map = geemap.Map()

## Load datasets

In [4]:
gm = ee.ImageCollection("GRIDMET/DROUGHT")
blm = ee.FeatureCollection("projects/ee-ericrjensen/assets/BLM_Allotments_All")
rapNPP = ee.ImageCollection("projects/rangeland-analysis-platform/npp-partitioned-v3")
rapCov = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v3")
mat = ee.ImageCollection("projects/rangeland-analysis-platform/gridmet-MAT")

## Pre-process data

In [5]:
# Subset gridMET by the years to subset
yearsToExport = 4
startDate = datetime.datetime(2019, 1, 1)
endDate = startDate + timedelta(days = (365.25*yearsToExport)-1)

### gridMET Drought Blends

In [6]:
# Define property list
property_list = ["system:index", "system:time_start"]

In [7]:
# Generate list of month starts for creating monthly drought blend Image Collection
monthList = pd.date_range(startDate,endDate, 
              freq='MS').strftime("%Y-%m-%d").tolist()
monthList[0:10]

['2019-01-01',
 '2019-02-01',
 '2019-03-01',
 '2019-04-01',
 '2019-05-01',
 '2019-06-01',
 '2019-07-01',
 '2019-08-01',
 '2019-09-01',
 '2019-10-01']

In [8]:
# Function to calculate short-term and long-term blends
def preprocess_gm_drought(img):

  # Define preliminary variables for short-term blend calculation
  stb_variable = "short_term_blend"
  stb_pdsi_img = img.select("pdsi")
  stb_z_img = img.select("z")
  stb_spi90d_img = img.select("spi90d")
  stb_spi30d_img = img.select("spi30d")

  # Define weights for short-term blend calculation
  stb_pdsi_coef = 0.2
  stb_z_coef = 0.35
  stb_spi90d_coef = 0.25
  stb_spi30d_coef = 0.2

  # Calculate short-term blend
  stblend = stb_pdsi_img.expression(
      "b() * pdsi_coef / 2 + spi90d * spi90d_coef + spi30d * spi30d_coef + z * z_coef / 2",{
          "spi90d": stb_spi90d_img, 
          "spi30d": stb_spi30d_img, 
          "z": stb_z_img, 
          "pdsi_coef": stb_pdsi_coef,
          "spi90d_coef": stb_spi90d_coef, 
          "spi30d_coef": stb_spi30d_coef, 
          "z_coef": stb_z_coef})

  # Define preliminary variables for long-term blend calculation
  ltb_variable = "long_term_blend"
  ltb_pdsi_img = img.select("pdsi")
  ltb_spi180d_img = img.select("spi180d")
  ltb_spi1y_img = img.select("spi1y")
  ltb_spi2y_img = img.select("spi2y")
  ltb_spi5y_img = img.select("spi5y")

  # Define weights for long-term blend calculation
  ltb_pdsi_coef = 0.35
  ltb_spi180d_coef = 0.15
  ltb_spi1y_coef = 0.2
  ltb_spi2y_coef = 0.2
  ltb_spi5y_coef = 0.1

  # Calculate short-term blend
  ltblend = ltb_pdsi_img.expression(
      "b() * pdsi_coef / 2 + spi180d* spi180d_coef + spi1y * spi1y_coef + spi2y * spi2y_coef + spi5y * spi5y_coef",{
          "spi180d": ltb_spi180d_img, 
          "spi1y": ltb_spi1y_img, 
          "spi2y": ltb_spi2y_img, 
          "spi5y": ltb_spi5y_img,
          "spi180d_coef": ltb_spi180d_coef, 
          "spi1y_coef": ltb_spi1y_coef, 
          "spi2y_coef": ltb_spi2y_coef,
          "spi5y_coef": ltb_spi5y_coef, 
          "pdsi_coef": ltb_pdsi_coef})
  
  return ltblend.addBands(stblend).select([0,1], [ltb_variable, stb_variable]).copyProperties(img, property_list)

In [9]:
# Map function to calculate drought blends over the subset years, convert to monthly median images, and convert to multi-band image
gm_blends = gm.map(calculate_droughtblends)

# Convert to monthly median image
def return_monthly_image(month):
  month = ee.Date(month)
  month_i = gm_blends.filterDate(month, month.advance(1, "month"))
  return(ee.Image(month_i.median()).set("system:index", month.format('YYYYMM')))

gm_blends_months = ee.ImageCollection(ee.List(monthList).map(return_monthly_image))

# Convert Image Collection to multi-band image
gm_blends_i = gm_blends_months.toBands()
# gm_blends_i.bandNames().getInfo()[0:5]

In [10]:
# Get separate time-series images for the two different blends
gm_ltb = gm_blends_i.select(['[0-9]{6}_long.*'])
gm_stb = gm_blends_i.select(['[0-9]{6}_short.*'])
# gm_ltb.bandNames().getInfo()[0:5]

### Rangeland Analysis Platform

In [11]:
# Create time-series images for RAP cover
rapCov_afg = rapCov.select("AFG").toBands()
rapCov_bgr = rapCov.select("BGR").toBands()
rapCov_ltr = rapCov.select("LTR").toBands()
rapCov_pfg = rapCov.select("PFG").toBands()
rapCov_shr = rapCov.select("SHR").toBands()
rapCov_tre = rapCov.select("TRE").toBands()
# rapCov_afg.bandNames().getInfo()[0:5]

In [12]:
# Create time-series images for RAP production
# Biomass conversion function
def biomassFunction(image):
    
    year = ee.Date(image.get('system:time_start')).format('YYYY')
    matYear = mat.filterDate(year).first()
    fANPP = (matYear.multiply(0.0129)).add(0.171).rename('fANPP') # fraction of NPP to allocate aboveground
  
    agb = image.multiply(0.0001).multiply(2.20462).multiply(4046.86).multiply(fANPP).multiply(2.1276)\
    .select(['afgNPP', 'pfgNPP'], ['AFG', 'PFG'])\
    .copyProperties(image, ['system:time_start'])\
    .set('year', year)

    herbaceous = ee.Image(agb).reduce(ee.Reducer.sum()).rename(['TOT'])
    
    agb = ee.Image(agb).addBands(herbaceous)

    return agb

In [13]:
# Apply function to annual image collection
rapPrd = rapNPP.map(biomassFunction);
rapPrd.first().bandNames().getInfo()

['AFG', 'PFG', 'TOT']

In [14]:
# Create time-series images
rapPrd_afg = rapPrd.select("AFG").toBands()
rapPrd_pfg = rapPrd.select("PFG").toBands()
rapPrd_tot = rapPrd.select("TOT").toBands()
rapPrd_afg.bandNames().getInfo()[0:5]

['1986_AFG', '1987_AFG', '1988_AFG', '1989_AFG', '1990_AFG']

## Reduce regions to time-series centroid Feature Collection

In [15]:
# # Subset input BLM allotment Feature Collection
# # Subset by state
# state = "NV"
# blm_sub = blm.filter(ee.Filter.eq("ADMIN_ST", state))

# # Subset by geometry
# geometry = ee.Geometry.Polygon([[[-109.4020, 39.7855], [-109.4020, 39.6080], [-109.1823, 39.6080], [-109.1823, 39.7855]]], None, False);
# blm_sub = blm.filterBounds(geometry)

# Use full Feature Collection
blm_sub = blm

In [16]:
# Function to return feature time-series as centroid feature collection
def imgs_to_pts(img, res, suffix, date_len):
  
  # Cast input image to ee.Image
  img = ee.Image(img)
  
  # Run reduce regions for allotments
  img_rr = img.reduceRegions(collection = blm_sub,\
                                 reducer = ee.Reducer.mean(),\
                                 scale = res,\
                                 tileScale = 16)
  
  # Function to clean up property names
  def clean_f(f):

    # Get uname for allotment
    f_uname = f.get("uname")
    
    # Rename uname for later sorting
    f = f.set("0_uname", f_uname)
    
    return(ee.Feature(f.select(['0_uname', '[0-9]{' + date_len + '}_' + suffix + '.*']))\
                       .centroid())

  # Apply function to clean up property names
  img_rr = img_rr.map(clean_f)

  # Get list of RR features
  img_rr_list = img_rr.toList(img_rr.size())

  # Get size of RR features
  img_rr_size = img_rr_list.size()
  
  # Function to create feature collection next to equator
  def pts_to_equator(i):
    
      i = ee.Number(i)
      
      # Get properties
      properties = ee.Feature(img_rr_list.get(i)).toDictionary()
      
      # Create geometry at equator
      geom = ee.Geometry.Point([i.multiply(0.002),0])
      
      # Return object
      return(ee.Feature(geom).set(properties))
  
  # Create equator feature collection
  proxy_fc = ee.FeatureCollection(ee.List.sequence(0, img_rr_size.subtract(1), 1).map(pts_to_equator))
    
  return(proxy_fc.set("VarName", suffix))

In [17]:
# Run function for gridMET drought blends
blm_sub_gm_ltb = imgs_to_pts(gm_ltb, 4000, "long_term_blend", '6')
blm_sub_gm_stb = imgs_to_pts(gm_stb, 4000, "short_term_blend", '6')
# ee.Feature(blm_sub_gm_ltb.first()).propertyNames().getInfo()[0:5]
# ee.Feature(blm_sub_gm_ltb.first()).getInfo()

In [18]:
# Run function for RAP cover
blm_sub_rapCov_afg = imgs_to_pts(rapCov_afg, 90, "AFG", '4')
blm_sub_rapCov_bgr = imgs_to_pts(rapCov_bgr, 90, "BGR", '4')
blm_sub_rapCov_ltr = imgs_to_pts(rapCov_ltr, 90, "LTR", '4')
blm_sub_rapCov_pfg = imgs_to_pts(rapCov_pfg, 90, "PFG", '4')
blm_sub_rapCov_shr = imgs_to_pts(rapCov_shr, 90, "SHR", '4')
blm_sub_rapCov_tre = imgs_to_pts(rapCov_tre, 90, "TRE", '4')
# ee.Feature(blm_sub_rapCov_afg.first()).propertyNames().getInfo()[0:5]

In [19]:
# Run function for RAP Production
blm_sub_rapPrd_afg = imgs_to_pts(rapPrd_afg, 90, "AFG", '4')
blm_sub_rapPrd_pfg = imgs_to_pts(rapPrd_pfg, 90, "PFG", '4')
blm_sub_rapPrd_tot = imgs_to_pts(rapPrd_tot, 90, "TOT", '4')
# ee.Feature(blm_sub_rapPrd_afg.first()).propertyNames().getInfo()[0:5]

## Convert centroid time-series to Image Collection time-series

In [20]:
# Function to convert centroid time-series feature collection to images
def pts_to_ics(fc, date_len):

  # Get variable name from Feature Collection 
  var_name = fc.get("VarName")
  
  # Get list of property names
  names = fc.first()\
          .propertyNames()\
          .sort()\
          .remove("system:index")

  # Function to convert features to images and attempt to convert to multiband images 
  def convert_to_img(property):
    
    # Get time-series date
    property_date = ee.String(property).slice(0, 8)
    
    # Reduce feature to image and set image properties
    img = fc.reduceToImage(properties = [property],\
                           reducer = ee.Reducer.sum())\
                  .set("system:index", property_date)\
                  .set("var_name", var_name)\
                  .rename([var_name])
                  
    return(img.updateMask(img.neq(0)))

  # Apply nested function to band names
  var_imgs = names.map(convert_to_img)
  
  return(ee.ImageCollection(var_imgs))

In [21]:
# Run function for gridMET drought blends
blm_sub_gm_ltb_ic = pts_to_ics(blm_sub_gm_ltb, 6)
blm_sub_gm_stb_ic = pts_to_ics(blm_sub_gm_stb, 6)

In [22]:
# Run function for RAP cover
blm_sub_rapCov_afg_ic = pts_to_ics(blm_sub_rapCov_afg, 4)
blm_sub_rapCov_bgr_ic = pts_to_ics(blm_sub_rapCov_bgr, 4)
blm_sub_rapCov_ltr_ic = pts_to_ics(blm_sub_rapCov_ltr, 4)
blm_sub_rapCov_pfg_ic = pts_to_ics(blm_sub_rapCov_pfg, 4)
blm_sub_rapCov_shr_ic = pts_to_ics(blm_sub_rapCov_shr, 4)
blm_sub_rapCov_tre_ic = pts_to_ics(blm_sub_rapCov_tre, 4)

In [23]:
# Run function for RAP Production
blm_sub_rapPrd_afg_ic = pts_to_ics(blm_sub_rapPrd_afg, 4)
blm_sub_rapPrd_pfg_ic = pts_to_ics(blm_sub_rapPrd_pfg, 4)
blm_sub_rapPrd_tot_ic = pts_to_ics(blm_sub_rapPrd_tot, 4)

In [24]:
# # Create export geometry
# geometry = blm_sub_rapCov_afg.geometry().bounds().buffer(1000)

# Map = geemap.Map()
# Map.addLayer(ee.Image(blm_sub_gm_ltb_ic.filter(ee.Filter.eq('system:index', '202201_l')).first()))
# Map.addLayer(ee.Image(blm_sub_gm_ltb_ic.filter(ee.Filter.eq('system:index', '0_uname')).first()))
# # Map.addLayer(geometry)
# Map

## Export Images to Image Collection Asset

In [72]:
# # Drought blends


# # -------- Define export parameters -----------

# # Define asset to export
# # List of potential assets
# # 1) blm_sub_gm_ltb_ic
# # 2) blm_sub_gm_stb_ic
# assetToExport = blm_sub_rapCov_afg_ic
# assetToExport_str = 'blm-rap-cover-afg'
# blend_suffix = "_s"

# # Parse Image Collection ID
# ic_id = f"projects/ce-datasets/assets/blm-allotment-ics/{assetToExport_str}"

# # Generate empty Image Collection asset to append images
# os.system(f"earthengine create collection {ic_id}")


# # -------- Get list of image IDs -----------

# # # # Get image ID
# # # Return from EE IC
# # img_ids = assetToExport.aggregate_array('system:index').getInfo()
# # print(img_ids)

# # Create image ID list for drought blends from monthlist
# img_ids = ["0_uname"] + [x.replace('-', '')[0:6]+suffix for x in monthList]


# # -------- Create export tasks ---------

# # Create export geometry
# geometry = blm_sub_gm_ltb.geometry().bounds().buffer(1000).bounds()

# # Loop over IDs to generate export tasks
# for i, img_id in enumerate(img_ids):
#     img = ee.Image(assetToExport.filter(ee.Filter.eq('system:index', img_id)).first())
#     task = ee.batch.Export.image.toAsset(
#         image = ee.Image(img),
#         description = img_id.replace(suffix, ''),
#         assetId = ic_id + "/" + img_id.replace(suffix, ''),
#         scale = 222.64,
#         region = geometry,
#         maxPixels = 1e13
#     )
    
#     task.start()
#     print(img_id.replace(suffix, ''))

0_uname
1986_AFG
1987_AFG
1988_AFG
1989_AFG
1990_AFG
1991_AFG
1992_AFG
1993_AFG
1994_AFG
1995_AFG
1996_AFG
1997_AFG
1998_AFG
1999_AFG
2000_AFG
2001_AFG
2002_AFG
2003_AFG
2004_AFG
2005_AFG
2006_AFG
2007_AFG
2008_AFG
2009_AFG
2010_AFG
2011_AFG
2012_AFG
2013_AFG
2014_AFG
2015_AFG
2016_AFG
2017_AFG
2018_AFG
2019_AFG
2020_AFG
2021_AFG


In [29]:
# Export RAP images

# -------- Define export parameters -----------

# Define asset to export
# List of potential assets
# 1) blm_sub_rapCov_afg_ic
# 2) blm_sub_rapCov_bgr_ic
# 3) blm_sub_rapCov_ltr_ic
# 4) blm_sub_rapCov_pfg_ic
# 5) blm_sub_rapCov_shr_ic
# 6) blm_sub_rapCov_tre_ic
# 7) blm_sub_rapPrd_afg_ic
# 8) blm_sub_rapPrd_pfg_ic
# 9) blm_sub_rapPrd_tot_ic
assetToExport = blm_sub_rapPrd_tot_ic
assetToExport_str = 'blm-rap-prod-tot'

# Define suffix of object for generating img ids
rap_suffix = "TOT"

# Parse Image Collection ID
ic_id = f"projects/ce-datasets/assets/blm-allotment-ics/{assetToExport_str}"

# Generate empty Image Collection asset to append images
# os.system(f"earthengine create collection {ic_id}")


# -------- Get list of image IDs -----------

# # Return from EE IC
# img_ids = assetToExport.aggregate_array('system:index').getInfo()
# print(img_ids)

# Create image ID list for RAP
x = range(1986, 2022, 1) 
img_ids = ["0_uname"] + [str(i) + "_" + rap_suffix for i in range(1986,2021+1)]


# -------- Create export tasks ---------

# Create export geometry
geometry = blm_sub_rapCov_afg.geometry().bounds().buffer(1000).bounds()

# Loop over IDs to generate export tasks
for i, img_id in enumerate(img_ids):

  img = ee.Image(assetToExport.filter(ee.Filter.eq('system:index', img_id)).first())

  task = ee.batch.Export.image.toAsset(
    image = ee.Image(img),
    description = img_id,
    assetId = ic_id + "/" + img_id,
    scale = 222.64,
    region = geometry,
    maxPixels = 1e13
  )

  task.start()
  
  print(img_id)

0_uname
1986_PFG
1987_PFG
1988_PFG
1989_PFG
1990_PFG
1991_PFG
1992_PFG
1993_PFG
1994_PFG
1995_PFG
1996_PFG
1997_PFG
1998_PFG
1999_PFG
2000_PFG
2001_PFG
2002_PFG
2003_PFG
2004_PFG
2005_PFG
2006_PFG
2007_PFG
2008_PFG
2009_PFG
2010_PFG
2011_PFG
2012_PFG
2013_PFG
2014_PFG
2015_PFG
2016_PFG
2017_PFG
2018_PFG
2019_PFG
2020_PFG
2021_PFG


In [76]:
!earthengine rm --recursive projects/ce-datasets/assets/blm-allotment-ics/blm-rap-cover-afg